In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
x_train = train.drop(['id', 'digit', 'letter'], axis=1).values
# x_train, x_train.shape

# x_train을 keras input에 넣기 위해 4차원으로 변경. 
# 28은 784개의 1차원벡터로 된 특성을 2차원으로 바꿔주기 위함. 784=28*28
x_train = x_train.reshape(-1, 28, 28, 1)
# type(x_train), np.max(x_train)
x_train = x_train/255

y = train['digit']
y_train = np.zeros((len(y), len(y.unique())))

# 원 핫 벡터로 
for i, digit in enumerate(y):
    y_train[i, digit] = 1

In [11]:
def create_cnn_model(x_train):
    inputs = tf.keras.layers.Input(x_train.shape[1:])

    bn = tf.keras.layers.BatchNormalization()(inputs)
    conv = tf.keras.layers.Conv2D(128, kernel_size=5, strides=1, padding='same', activation='relu')(bn)
    bn = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)
    dropout=tf.keras.layers.Dropout(rate=0.5)(pool)

    bn = tf.keras.layers.BatchNormalization()(dropout)
    conv = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    bn = tf.keras.layers.BatchNormalization()(conv)
    conv = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    pool = tf.keras.layers.MaxPooling2D((2, 2))(conv)
    dropout=tf.keras.layers.Dropout(rate=0.5)(pool)

    flatten = tf.keras.layers.Flatten()(dropout)

    bn = tf.keras.layers.BatchNormalization()(flatten)
    dense = tf.keras.layers.Dense(1000, activation='relu')(bn)
    dropout=tf.keras.layers.Dropout(rate=0.3)(dense)

    bn = tf.keras.layers.BatchNormalization()(dropout)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(bn)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

    return model

In [12]:
model = create_cnn_model(x_train)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
64/64 [==============================] - 30s 463ms/step - loss: 2.5680 - accuracy: 0.3481
Epoch 2/20
64/64 [==============================] - 31s 483ms/step - loss: 1.6216 - accuracy: 0.5059
Epoch 3/20
64/64 [==============================] - 30s 466ms/step - loss: 1.2382 - accuracy: 0.5972
Epoch 4/20
64/64 [==============================] - 31s 478ms/step - loss: 0.9567 - accuracy: 0.6899
Epoch 5/20
64/64 [==============================] - 24s 370ms/step - loss: 0.8060 - accuracy: 0.7227
Epoch 6/20
64/64 [==============================] - 23s 366ms/step - loss: 0.7187 - accuracy: 0.7510
Epoch 7/20
64/64 [==============================] - 27s 422ms/step - loss: 0.5731 - accuracy: 0.8032
Epoch 8/20
64/64 [==============================] - 30s 473ms/step - loss: 0.5588 - accuracy: 0.8091
Epoch 9/20
64/64 [==============================] - 27s 425ms/step - loss: 0.4586 - accuracy: 0.8350
Epoch 10/20
64/64 [==============================] - 28s 444ms/step - loss: 0.3987 - accura

In [13]:
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255
y=model.predict(x_test)

In [14]:
# y.shape

In [15]:
digit=np.argmax(y,axis=1)
# number

In [16]:
pd_submisson=pd.DataFrame({"digit":digit},index=test["id"])
# pd_submisson.index

In [17]:
# pd_submisson

In [18]:
pd_submisson.to_csv("cho_try1.csv", encoding="utf-8")